In [1]:
import torch
from torch.utils.data import DataLoader
from models.resnet_clothing_model import ClothingClassifier
from data.dataset import GarmentDataset
from utils.loss import compute_loss
from utils.metrics import accuracy
from data.transformation import CustomResNetTransform
from tqdm import tqdm
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.model_selection import train_test_split

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""
DATA SET UP
"""
df = pd.read_csv('dataset/list_category_img.txt', delim_whitespace=True, skiprows=1)
selected_labels = [3, 6, 11, 16, 17, 18, 19, 26, 32, 33, 41]
selected_names = ["Blouse", "Cardigan", "Jacket", "Sweater", "Tank", "Tee", "Top", "Jeans", "Shorts", "Skirts", "Dress"]
df = df[df['category_label'].isin(selected_labels)]
df = df.reset_index(drop=True)


label_mapping = {original: new for new, original in enumerate(selected_labels)}
df['category_label'] = df['category_label'].map(label_mapping)

train_val_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=1331,
    stratify=df['category_label']
)
train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.25,
    random_state=1331,
    stratify=train_val_df['category_label']
)

balanced_train_df = train_df.groupby('category_label', group_keys=False).apply(
    lambda x: x.sample(n=1800, random_state=1331)
).reset_index(drop=True)

minority_classes = [2, 6, 7, 3, 1, 9, 4, 8]
transform_strong = CustomResNetTransform(strong_aug=True)
transform_normal = CustomResNetTransform(strong_aug=False)

train_dataset = GarmentDataset(train_df, minority_classes, transform_strong, transform_normal)
balanced_dataset = GarmentDataset(balanced_train_df, minority_classes, transform_strong, transform_normal)
val_dataset = GarmentDataset(val_df, minority_classes, transform_strong, transform_normal)
test_dataset = GarmentDataset(test_df, minority_classes, transform_strong, transform_normal)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
balanced_loader = DataLoader(balanced_dataset, batch_size=64, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=8)

# Get class weights for loss function
class_sample_counts = train_df['category_label'].value_counts().sort_index().tolist()
class_counts_tensor = torch.tensor(class_sample_counts, dtype=torch.float)
class_weights = 1.0 / class_counts_tensor
class_weights = class_weights / class_weights.mean()

/var/folders/nj/y687jr_s3vdc8l49m6_mn7sm0000gn/T/ipykernel_45429/1674405879.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('dataset/list_category_img.txt', delim_whitespace=True, skiprows=1)
/var/folders/nj/y687jr_s3vdc8l49m6_mn7sm0000gn/T/ipykernel_45429/1674405879.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_train_df = train_df.groupby('category_label', group_keys=False).apply(


In [3]:
"""
TRAIN CODE
"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

NUM_CLASSES = 11
FROZEN_LAYERS = 60
LEARNING_RATE = 0.001
NUM_EPOCHS = 22

model = ClothingClassifier(num_classes=NUM_CLASSES, num_frozen_resnet_layers=FROZEN_LAYERS).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.1, patience=3, verbose=True
)


epoch_losses = []
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0

    # 3) Training loop
    with tqdm(train_loader, desc=f"Epoch Train {epoch+1}/{NUM_EPOCHS}", unit="batch") as pbar:
        for images, category_id in pbar:
            images = images.to(device)
            category_id = category_id.to(device)


            optimizer.zero_grad()
            preds = model(images)
            loss = compute_loss(preds, {'category_id': category_id}, class_weights=class_weights.to(device))

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            batch_correct = accuracy(preds, category_id)
            total_correct += batch_correct
            total_samples += images.size(0)

            pbar.set_postfix(loss=loss.item())

    # 4) Check against a validator to adjust learning rate
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        with tqdm(val_loader, desc=f"Epoch Val {epoch+1}/{NUM_EPOCHS}", unit="batch") as pbar:
            for images, category_id in pbar:
                images = images.to(device)
                category_id = category_id.to(device)
                preds = model(images)
                loss = compute_loss(preds, {'category_id': category_id})
                val_loss += loss.item()
    val_loss /= len(val_loader)
    scheduler.step(val_loss)
    epoch_loss = running_loss / len(train_loader)
    epoch_losses.append(epoch_loss)
    epoch_accuracy = (total_correct / total_samples) * 100
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}: Accuracy = {epoch_accuracy:.2f}%, Training Loss = {epoch_loss:.4f}, Validation Loss = {val_loss:.4f}")
print("Training complete. Saving model...")
torch.save(model.state_dict(), 'clothing_classifier.pth')
print("Model saved.")


plt.subplot(1, 1, 1)
plt.plot(epoch_losses, marker='o', color='red', label='Epoch Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Across Epochs')
plt.legend()

plt.tight_layout()
plt.show()

Using device: cpu


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
